In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')

In [3]:
# Verificación inicial
print(f"Sell-In: {sellin.shape[0]} filas y {sellin.shape[1]} columnas")
print(f"Productos: {productos.shape[0]} filas y {productos.shape[1]} columnas")
print(f"Stocks: {stocks.shape[0]} filas y {stocks.shape[1]} columnas")

Sell-In: 2945818 filas y 7 columnas
Productos: 1262 filas y 6 columnas
Stocks: 13691 filas y 3 columnas


In [4]:
# 3. MERGE INICIAL
df = sellin.merge(productos, on="product_id", how="left")
df = df.merge(stocks, on=["product_id", "periodo"], how="left")
print(f"Ventas-Productos-Stocks: {df.shape[0]} filas y {df.shape[1]} columnas")

Ventas-Productos-Stocks: 2988650 filas y 13 columnas


In [5]:
productos_clean = productos.drop_duplicates(subset=['product_id'], keep='first')
print(productos_clean.shape)

(1251, 6)


In [6]:
df = sellin.merge(productos_clean, on="product_id", how="left")
df = df.merge(stocks, on=["product_id", "periodo"], how="left")
print(sellin.shape)
print(df.shape)

(2945818, 7)
(2945818, 13)


In [7]:
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

# Armado del Dataset

In [8]:
# Convertimos periodo a datetime
df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")

# Determinar vida útil de cada producto (primer y último periodo)
vida_producto = df.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()

# Expandimos cada producto con todos los periodos de su vida útil
periodos_producto = []
for _, row in vida_producto.iterrows():
    periodos = pd.date_range(start=row["min"], end=row["max"], freq="MS")
    for p in periodos:
        periodos_producto.append((p, row["product_id"]))

df_producto_periodo = pd.DataFrame(periodos_producto, columns=["periodo_dt", "product_id"])



# Agregar columna periodo en formato AAAAMM
df_producto_periodo["periodo"] = df_producto_periodo["periodo_dt"].dt.strftime("%Y%m").astype(int)

df_producto_periodo.drop(columns=['periodo_dt'],inplace=True)
# Ordenar por periodo_dt (ascendente) y luego por product_id
df_producto_periodo = df_producto_periodo.sort_values(by=["product_id", "periodo"], ascending=True).reset_index(drop=True)



###########
toneladas_vendidas = df.copy()
# Agregar los datos por periodo y product_id para obtener la serie temporal
# Sumamos tn, cust_request_qty y cust_request_tn por periodo y product_id
toneladas_vendidas = df.groupby(['periodo', 'product_id']).agg({
    'tn': 'sum',
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum'
}).reset_index()

# Paso 5: Unir con las toneladas efectivamente vendidas (tn)
df_merge = df_producto_periodo.merge(toneladas_vendidas[["periodo", "product_id", "tn"]],
                             on=["periodo", "product_id"],
                             how="left")

print(df_merge.shape)
df_merge["tn"] = df_merge["tn"].fillna(0)

(31522, 3)


# Prophet

In [9]:
df_merge.head()

,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132


# Prophet

In [14]:
productos_a_predecir = pd.read_csv("datasets/product_id_apredecir201912.csv")
# productos_a_predecir = list(productos_a_predecir['product_id'])

In [21]:
import pandas as pd
from prophet import Prophet

df = df_merge.copy()

# Asegurar que el campo periodo esté en formato datetime (Prophet espera columna 'ds' de fechas)
df['ds'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')
df = df.rename(columns={'tn': 'y'})

# 📤 2. Filtrar solo las columnas necesarias
df = df[['product_id', 'ds', 'y']]

# 📅 Fecha objetivo
fecha_prediccion = pd.to_datetime('2020-02-01')

# 📈 3. Crear diccionario para almacenar predicciones
predicciones = []

# 4️⃣ Recorrer cada producto de la lista (750 productos)
for product_id in productos_a_predecir['product_id']:
    # Filtrar datos de este producto
    df_prod = df[df['product_id'] == product_id]
    
    if df_prod.empty or len(df_prod) < 3:
        # Si no hay datos o son insuficientes para entrenar, guardar NaN
        predicciones.append({'product_id': product_id, 'periodo': 202002, 'tn_pred': None})
        continue
    
    # Preparar datos para Prophet
    df_prophet = df_prod[['ds', 'y']].sort_values('ds')
    
    # Entrenar Prophet
    m = Prophet()
    m.fit(df_prophet)
    
    # Crear dataframe para fechas futuras
    future = pd.DataFrame({'ds': [fecha_prediccion]})
    
    # Hacer predicción
    forecast = m.predict(future)
    y_pred = forecast.loc[forecast['ds'] == fecha_prediccion, 'yhat'].values[0]
    
    # Guardar predicción
    predicciones.append({'product_id': product_id, 'periodo': 202002, 'tn_pred': y_pred})

# 📈 5. Convertir resultados a DataFrame
df_predicciones = pd.DataFrame(predicciones)

# 📤 Mostrar resultados
print(df_predicciones)


AttributeError: 'Prophet' object has no attribute 'stan_backend'

In [20]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable
